In [4]:
import requests
from bs4 import BeautifulSoup
import csv

def scrape_kingfisher():
    category_urls = [
        "https://kingfisher.kz/ryba/",
        "https://kingfisher.kz/ikra/",
        "https://kingfisher.kz/myaso/",
        "https://kingfisher.kz/antipasti/",
        "https://kingfisher.kz/bakaleya/",
        "https://kingfisher.kz/sneki-i-orehi/",
        "https://kingfisher.kz/maslo/",
        "https://kingfisher.kz/cheeses/",
    ]
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36"
    }
    
    all_products = []
    
    for link in category_urls:
        all_products.extend(extract_category_data(link, headers))
    
    write_to_csv(all_products, "kingfisher.csv")


def extract_category_data(category_url, headers):
    """Fetch and parse product data from a single category page."""
    try:
        response = requests.get(category_url, headers=headers)
        response.raise_for_status()
        
        soup = BeautifulSoup(response.content, 'html.parser')
        category_title = soup.find("h1", class_="title").text.strip() if soup.find("h1", class_="title") else "Unknown"
        
        product_blocks = soup.select("div.goodsBlock")
        category_products = []
        
        for block in product_blocks:
            try:
                product_name = block.select_one("a.title span").text.strip() if block.select_one("a.title span") else "Unnamed"
                product_price = block.select_one("span.price span.new").text.strip() if block.select_one("span.price span.new") else "Price not available"
                city = "Astana"
                
                category_products.append({
                    "Product Name": product_name,
                    "Category": category_title,
                    "Price": product_price,
                    "City": city
                })
            except Exception as e:
                print(f"Error parsing product block: {e}")
        return category_products
    except requests.RequestException as e:
        print(f"Failed to load category page {category_url}: {e}")
        return []


def write_to_csv(data, filename):
    """Save product data to a CSV file."""
    headers = ["Product Name", "Category", "Price", "City"]
    try:
        with open(filename, mode="w", encoding="utf-8", newline="") as csvfile:
            writer = csv.DictWriter(csvfile, fieldnames=headers)
            writer.writeheader()
            writer.writerows(data)
        print(f"Data successfully written to {filename}")
    except IOError as e:
        print(f"Error writing to file: {e}")


if __name__ == "__main__":
    scrape_kingfisher()


Data successfully written to kingfisher.csv
